In [2]:
import sys
import os
from scipy.io import wavfile
import numpy as np
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.init import *
from lasagne.nonlinearities import *
from lasagne.objectives import *
from lasagne.updates import *
import cPickle as pickle
import matplotlib.pyplot as plt

In [5]:
os.environ["DATA_DIR"] = "/Volumes/CB_RESEARCH/vocal_synthesis"

In [6]:
data_folder = os.environ["DATA_DIR"]
num_inputs = 800
#seq_length = 50

In [7]:
with open(data_folder + os.path.sep + "x0.05_5_all_use_mean.pkl") as f:
    dat = pickle.load(f)

In [8]:
X_train, X_valid, X_test = dat[0]

In [9]:
X_train.shape, X_valid.shape, X_test.shape

((1742, 100, 800), (217, 100, 800), (217, 100, 800))

In [58]:
num_inputs, num_hidden, num_classes = 800, 10, 800
l_input = InputLayer((None, None, num_inputs))
l_forward = LSTMLayer(l_input, num_units=num_hidden)
n_batch, n_time_steps, _ = l_input.input_var.shape
l_reshape = ReshapeLayer(l_forward, (-1, num_hidden))
l_dense = DenseLayer(l_reshape, num_units=num_classes, nonlinearity=linear)
l_out = ReshapeLayer(l_dense, (n_batch, n_time_steps, num_classes))

In [54]:
"""

>>> from lasagne.layers import *
>>> num_inputs, num_units, num_classes = 10, 12, 5
>>> # By setting the first two dimensions as None, we are allowing them to vary
>>> # They correspond to batch size and sequence length, so we will be able to
>>> # feed in batches of varying size with sequences of varying length.
>>> l_inp = InputLayer((None, None, num_inputs))
>>> # We can retrieve symbolic references to the input variable's shape, which
>>> # we will later use in reshape layers.
>>> batchsize, seqlen, _ = l_inp.input_var.shape
>>> l_lstm = LSTMLayer(l_inp, num_units=num_units)
>>> # In order to connect a recurrent layer to a dense layer, we need to
>>> # flatten the first two dimensions (our "sample dimensions"); this will
>>> # cause each time step of each sequence to be processed independently
>>> l_shp = ReshapeLayer(l_lstm, (-1, num_units))
>>> l_dense = DenseLayer(l_shp, num_units=num_classes)
>>> # To reshape back to our original shape, we can use the symbolic shape
>>> # variables we retrieved above.
>>> l_out = ReshapeLayer(l_dense, (batchsize, seqlen, num_classes))

"""

'\n\n>>> from lasagne.layers import *\n>>> num_inputs, num_units, num_classes = 10, 12, 5\n>>> # By setting the first two dimensions as None, we are allowing them to vary\n>>> # They correspond to batch size and sequence length, so we will be able to\n>>> # feed in batches of varying size with sequences of varying length.\n>>> l_inp = InputLayer((None, None, num_inputs))\n>>> # We can retrieve symbolic references to the input variable\'s shape, which\n>>> # we will later use in reshape layers.\n>>> batchsize, seqlen, _ = l_inp.input_var.shape\n>>> l_lstm = LSTMLayer(l_inp, num_units=num_units)\n>>> # In order to connect a recurrent layer to a dense layer, we need to\n>>> # flatten the first two dimensions (our "sample dimensions"); this will\n>>> # cause each time step of each sequence to be processed independently\n>>> l_shp = ReshapeLayer(l_lstm, (-1, num_units))\n>>> l_dense = DenseLayer(l_shp, num_units=num_classes)\n>>> # To reshape back to our original shape, we can use the symbo

----

In [71]:
X = l_input.input_var
net_out = get_output(l_out, X, deterministic=True)
get_out = theano.function([X], net_out)

In [72]:
loss = squared_error(net_out[:,0:-1,:], X[:,1::,:])

In [74]:
loss_fn = theano.function([X], loss)

In [75]:
loss_fn(X_train).mean()

0.014322396843309694

In [76]:
eval_fn = theano.function([X], get_output(l_out, X))

In [79]:
eval_fn(X_train[0:1,0:5]).shape

(1, 5, 800)

In [101]:
((eval_fn(X_train[0:5])[:,0:-1,:] - X_train[0:5,1::,:])**2).mean(axis=(1,2)).mean()

0.012689432842499635

In [98]:
((eval_fn(X_train[0:5])[:,0:-1,:] - X_train[0:5,1::,:])**2).mean()

0.012689432842499635

In [48]:
train = theano.function([X], loss, updates=updates)

In [102]:
batch_size = 64
for epoch in range(0, 10):
    b = 0
    train_losses = []
    while True:
        if b*batch_size >= X_train.shape[0]:
            break
        X_batch = X_train[b*batch_size : (b+1)*batch_size]
        train_losses.append( train(X_batch) )
        b += 1
    print "train loss this epoch: %f" % np.mean(train_losses)
    print "  valid loss this epoch: %f" % loss_fn(X_valid)

train loss this epoch: 0.001053
  valid loss this epoch: 0.000863
train loss this epoch: 0.000708
  valid loss this epoch: 0.000633
train loss this epoch: 0.000542
  valid loss this epoch: 0.000505
train loss this epoch: 0.000443
  valid loss this epoch: 0.000423
train loss this epoch: 0.000379
  valid loss this epoch: 0.000368
train loss this epoch: 0.000334
  valid loss this epoch: 0.000329
train loss this epoch: 0.000302
  valid loss this epoch: 0.000300
train loss this epoch: 0.000278
  valid loss this epoch: 0.000278
train loss this epoch: 0.000259
  valid loss this epoch: 0.000259
train loss this epoch: 0.000241
  valid loss this epoch: 0.000242


In [26]:
get_out(X_valid[0:1]).shape

(1, 500, 1)

In [103]:
trained_model = lasagne.layers.get_all_param_values(l_out_500)

---

In [95]:
test_seq_length = 1
l_input = InputLayer((None, test_seq_length, num_inputs))
l_forward = RecurrentLayer(l_input, num_units=10)
l_shp = ReshapeLayer(l_forward, (-1, 10))
l_dense = DenseLayer(l_shp, num_units=1, nonlinearity=linear)
l_out_1 = ReshapeLayer(l_dense, (-1, test_seq_length, 1))

In [104]:
lasagne.layers.set_all_param_values(l_out_1, trained_model)

In [105]:
eval_fn = theano.function([X], get_output(l_out_1, X))

In [106]:
seed = X_valid[0:1]
seed = seed.reshape( (seed.shape[1], 1, 1) )
seed = seed[0].reshape( (1,1,1) )

In [107]:
generated_seq = []
prev_input = seed
for x in range(0, 100000):
    next_input = eval_fn(prev_input)
    generated_seq.append(next_input.flatten()[0])
    prev_input = next_input

In [108]:
generated_seq

[0.80846236756157741,
 1.0128761148231722,
 1.2747363396885296,
 1.6101872488623801,
 2.0399100619525381,
 2.5903980500307471,
 3.295589898793069,
 4.1989620964261443,
 5.3562093460770184,
 6.8386782557173689,
 8.7377659992512342,
 11.170555135122889,
 14.287031980264311,
 18.279333566576156,
 23.393593273307523,
 29.945115442267472,
 38.337814521252319,
 49.089117197768324,
 62.861862789409599,
 80.505168613773492,
 103.1067797697186,
 132.06013079870354,
 169.15025371178263,
 216.6638287839784,
 277.53016296765696,
 355.50178752611242,
 455.38580909275026,
 583.34027741147736,
 747.25384142236658,
 957.23210024976538,
 1226.2206336041925,
 1570.8031226686487,
 2012.2237672185959,
 2577.6970330363561,
 3302.0854781681346,
 4230.0490993688345,
 5418.7987102426869,
 6941.623102350085,
 8892.4074458188243,
 11391.421497896321,
 14592.734474079154,
 18693.713723458251,
 23947.192820616496,
 30677.059261286046,
 39298.222774385293,
 50342.195336818397,
 64489.859950121616,
 82613.448439589

-----

----

In [18]:
data_folder = os.environ["DATA_DIR"]
in_file = data_folder + os.path.sep + "x1_50_all.pkl"

In [39]:
with open(in_file) as f:
    dat = pickle.load(f)
X_train, X_valid, X_test = dat[0]

num_inputs = X_train.shape[2]

sys.stderr.write("X_train shape = %s\n" % str(X_train.shape))
sys.stderr.write("X_valid shape = %s\n" % str(X_valid.shape))
sys.stderr.write("X_test shape = %s\n" % str(X_test.shape))

args = dict()
args["seed"] = 0
args["batch_size"] = 16
args["learning_rate"] = 0.01
args["momentum"] = 0.9
args["num_epochs"] = 2000
args["X_train"] = X_train
args["X_valid"] = X_valid
args["X_test"] = X_test
args["update_method"] = rmsprop

args["config"] = "19feb_testing_d.py"

X_train shape = (174, 50, 16000)
X_valid shape = (21, 50, 16000)
X_test shape = (21, 50, 16000)


In [40]:
import rnn_experiment as exp
reload(exp)
symbols = exp.prepare(args)
out_fn = symbols["out_fn"]
l_out = symbols["l_out"]
with open(data_folder + os.path.sep + "expensive_models" + os.path.sep + "22feb_testing_a_rmsprop.model") as f:
    model = pickle.load(f)
set_all_param_values(l_out, model)

loading config: configurations/19feb_testing_d.py
Number of params in model: 55232200


In [41]:
X_train.shape, X_valid.shape, X_test.shape

((174, 50, 16000), (21, 50, 16000), (21, 50, 16000))

In [43]:
l_out.shape

(-1, 50, 16000)

In [44]:
args["X_train"][0:1].shape

(1, 50, 16000)

In [45]:
out_fn( args["X_train"][0:1] ).shape

(1, 50, 16000)

In [46]:
seq_length, num_inputs

(50, 16000)

In [50]:
X_train[0:1].shape

(1, 50, 16000)

In [70]:
generated_seq = X_train[0:1].flatten().tolist()
prev_seq_chunk = X_train[0:1]
for x in range(0, 100):
    next_seq_chunk = out_fn(prev_seq_chunk) # x2, ..., x1001
    generated_seq += next_seq_chunk[0][-1].flatten().tolist() # append x1001 to end
    prev_seq_chunk = next_seq_chunk
    

In [71]:
float(len(generated_seq)) / 16000.0

150.0

In [78]:
fs, data = wavfile.read(data_folder + os.path.sep + "data.wav")
data = np.asarray(data, dtype="float32")

In [80]:
this_min, this_max = np.min(data), np.max(data)

In [83]:
this_min, this_max

(-8936.0, 9124.0)

In [81]:
tmp = np.asarray(generated_seq)
tmp = (tmp * (this_max - this_min)) + this_min

In [84]:
np.max(tmp), np.min(tmp)

(8128.0001094341278, -8355.9999829381704)

In [85]:
wavfile.write(filename="/tmp/t.wav", rate=16000, data=tmp)

In [74]:
plt.plot(generated_seq)
plt.show()

----

In [13]:
generated_seq = [args["X_train"][0:1].flatten().tolist()]
prev_seq_chunk = args["X_train"][0:1]
ptr = 0
for x in range(0, 5):
    next_seq_chunk = out_fn(prev_seq_chunk) # x2, ..., x1001
    generated_seq.append(next_seq_chunk.flatten()[-1]) # append x1001 to end
    ptr += 1
    prev_seq_chunk = np.asarray(generated_seq[ptr:ptr+seq_length]).reshape( (1, seq_length, 1) )

ValueError: ('shapes (50,1) and (1600,6000) not aligned: 1 (dim 1) != 1600 (dim 0)', (50, 1), (1600, 6000))
Apply node that caused the error: Dot22(Reshape{2}.0, Reshape{2}.0)
Toposort index: 39
Inputs types: [TensorType(float64, matrix), TensorType(float64, matrix)]
Inputs shapes: [(50, 1), (1600, 6000)]
Inputs strides: [(8, 8), (48000, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{3}(Dot22.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [163]:
plt.plot(generated_seq)
plt.show()

In [162]:
len(generated_seq)

1098